In [1]:
# change working directory to the '/Users/hannesleonhard/Documents/BachelorWInfo/Robustness-SDC/PerturbationDrive/'
import os
os.chdir('/Users/hannesleonhard/Documents/BachelorWInfo/Robustness-SDC/PerturbationDrive/')
os.getcwd()

'/Users/hannesleonhard/Documents/BachelorWInfo/Robustness-SDC/PerturbationDrive'

In [2]:
from perturbationdrive import CustomRoadGenerator
from perturbationdrive.RoadGenerator.Roads.road_visualizer import visualize_road

road_gen = CustomRoadGenerator()

# road 1 is straight
road1_angles = [0, 0, 0, 0, 0, 0, 0, 0]
road1_segments = [25, 25, 25, 25, 25, 25, 25, 25]

# road 2-10 are a set of rooads with different smoothnes and turns
road2_angles = [0, -20, -10, -11, -13, -25, -24, 4]
road2_segments = [25, 25, 25, 25, 25, 25, 25, 25]

road3_angles = [0, 24, 24, -5, -7, -24, -20, 17]
road3_segments = [25, 25, 25, 25, 25, 25, 25, 25]

road4_angles = [0, 6, 8, 20, -3, 20, -13, -27]
road4_segments = [25, 25, 25, 25, 25, 25, 25, 25]

road5_angles = [0, -2, -30, -8, 2, 2, 4, -30]
road5_segments = [25, 25, 25, 25, 25, 25, 25, 25]

road6_angles = [0, 0, 27, 35, -35, -35, -6, -35]
road6_segments = [25, 25, 25, 25, 25, 25, 25, 25]

road7_angles = [0, -27, -20, 35, 23, -3, 4, -35]
road7_segments = [25, 25, 25, 25, 25, 25, 25, 25]

road8_angles = [0, 30, 30, -30, 32, -32, 25, -12]
road8_segments = [25, 25, 25, 25, 25, 25, 25, 25]

road9_angles = [0, -35, -20, -35, 35, 35, 35, -35]
road9_segments = [25, 25, 25, 25, 25, 25, 25, 25]

road10_angles = [0, -35, 0, -17, -35, 35, 6, -22]
road10_segments = [25, 25, 25, 25, 25, 25, 25, 25]

angles = [
    #road1_angles,
    #road2_angles,
    #road3_angles,
    #road4_angles,
    #road5_angles,
    #road6_angles,
    road7_angles,
    road8_angles,
    road9_angles,
    road10_angles,
]
segments = [
    #road1_segments,
    #road2_segments,
    #road3_segments,
    #road4_segments,
    #road5_segments,
    #road6_segments,
    road7_segments,
    road8_segments,
    road9_segments,
    road10_segments,
]

/Users/hannesleonhard/micromamba/envs/donkey_opensbt/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/Users/hannesleonhard/micromamba/envs/donkey_opensbt/lib/python3.9/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.13.0 and strictly below 2.16.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.9.2 and is not supported. 
Some things might work, some things might not.
If you

pygame 2.5.2 (SDL 2.28.3, Python 3.9.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
# setup a sdsandbox simulator
from examples.self_driving_sandbox_donkey.sdsandbox_simulator import SDSandboxSimulator

simulator_sdsandbox = SDSandboxSimulator(
    simulator_exe_path="./examples/self_driving_sandbox_donkey/sim/donkey-sim.app",
    host="127.0.0.1",
    port=9091,
    show_image_cb=False,
)

In [4]:
from perturbationdrive import PerturbationDrive
from examples.models.dave2_agent import Dave2Agent
from examples.models.epoch_agent import EpochAgent
from examples.models.chauffeur_agent import ChauffeurAgent

dave_aug = Dave2Agent(model_path="./examples/models/dave_static_aug.h5")
dave_aug.name = "FILTER-AUG-DAVE2"
agents = [dave_aug]

dynamic_perturbations = [
    "dynamic_snow_filter",
    "dynamic_rain_filter",
    "dynamic_object_overlay",
    "dynamic_sun_filter",
    "dynamic_lightning_filter",
    "dynamic_smoke_filter",
    "static_snow_filter",
    "static_rain_filter",
    "static_object_overlay",
    "static_sun_filter",
    "static_lightning_filter",
    "static_smoke_filter",
]

Using model: ./examples/models/dave_static_aug.h5


In [5]:
from perturbationdrive import Scenario

try:
    benchmark = PerturbationDrive(simulator=simulator_sdsandbox, ads=dave_aug)
    benchmark.simulator.connect()
    starting_pos = benchmark.simulator.initial_pos

    # init all scenarios
    scenarios = []
    counter = 6
    # iterate over all agents
    for agent in agents:
        for angle, segment in zip(angles, segments):
            # run all perturbations for each road
            waypoints = road_gen.generate(
                starting_pos=starting_pos, angles=angle, seg_length=segment
            )
            scenarios = []
            for func in dynamic_perturbations:
                for i in range(5):
                    scenarios.append(
                        Scenario(
                            waypoints=waypoints,
                            perturbation_function=func,
                            perturbation_scale=i,
                        )
                    )
            benchmark.setADS(agent)
            print(f"Starting Agent {agent.name} on road {angle}")
            # run the benchmark
            res = benchmark.simulate_scenarios(
                scenarios=scenarios,
                log_dir=f"./logs/donkey_benchmark_sVd_perturbations_road{counter}_{agent.name}.json",
            )
            counter += 1
            print(f"Finished Agent {agent.name} on road {angle}")
except Exception as e:
    print(e)
finally:
    benchmark.simulator.tear_down()

INFO:DonkeyProcess:This is the launch string /Users/hannesleonhard/Documents/BachelorWInfo/Robustness-SDC/PerturbationDrive/./examples/self_driving_sandbox_donkey/sim/donkey-sim.app/Contents/MacOS/Simulator


[UnityMemory] Configuration Parameters - Can be set up in boot.config
    "memorysetup-bucket-allocator-granularity=16"
    "memorysetup-bucket-allocator-bucket-count=8"
    "memorysetup-bucket-allocator-block-size=4194304"
    "memorysetup-bucket-allocator-block-count=1"
    "memorysetup-main-allocator-block-size=16777216"
    "memorysetup-thread-allocator-block-size=16777216"
    "memorysetup-gfx-main-allocator-block-size=16777216"
    "memorysetup-gfx-thread-allocator-block-size=16777216"
    "memorysetup-cache-allocator-block-size=4194304"
    "memorysetup-typetree-allocator-block-size=2097152"
    "memorysetup-profiler-bucket-allocator-granularity=16"
    "memorysetup-profiler-bucket-allocator-bucket-count=8"
    "memorysetup-profiler-bucket-allocator-block-size=4194304"
    "memorysetup-profiler-bucket-allocator-block-count=1"
    "memorysetup-profiler-allocator-block-size=16777216"
    "memorysetup-profiler-editor-allocator-block-size=1048576"
    "memorysetup-temp-allocator-siz

Exception: [Errno 54] Connection reset by peer
CRITICAL:DonkeySimMsgHandler:DonkeySimMsgHandler: +++++ Warning: Donkey Sim Aborted +++++
INFO:DonkeySimMsgHandler:DonkeySimMsgHandler: +++++ Warning: Donkey Sim Message Stoped +++++


In [ ]:
import zipfile

for agent in agents:
    files = [
        f"./logs/donkey_benchmark_sVd_perturbations_road{counter}_{agent.name}.json"
        for counter in range(10)
    ]
    # Create a ZipFile object
    with zipfile.ZipFile(f"./logs/donkey_benchmark_sVd_perturbations_{agent.name}.zip", 'w') as zipf:
        # Add each file to the zip file
        for file in files:
            zipf.write(file)